In [1]:
# if hit plotting library issues, try resetting LD path for julia
# can set in ~/.local/share/jupyter/kernels/
# @assert ENV["LD_LIBRARY_PATH"] == ""
import Pkg
pkgs = [
    "Revise",
    "DataFrames",
    "uCSV",
    "StatsBase",
    "Statistics",
    "Dates",
    "FASTX"
]
# Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end
# Pkg.develop(path="/global/cfs/projectdirs/m4269/cjprybol/Mycelia")
# Pkg.develop(path="../../..")
import Mycelia

[ Info: Precompiling Mycelia [453d265d-8292-4a7b-a57c-dce3f9ae6acd]
[ Info: Skipping precompilation since __precompile__(false). Importing Mycelia [453d265d-8292-4a7b-a57c-dce3f9ae6acd].


In [2]:
PROJECT_BASEDIR = dirname(pwd())
ncbi_metadata = Mycelia.load_refseq_metadata()

filtered_ncbi_metadata = 
ncbi_metadata[
    (ncbi_metadata[!, "excluded_from_refseq"] .== "na") .&
    (ncbi_metadata[!, "genome_rep"] .== "Full") .&
    map(x -> x in Set(["Chromosome", "Complete Genome"]), ncbi_metadata[!, "assembly_level"]) .&
    map(x -> x in Set(["reference genome", "representative genome"]), ncbi_metadata[!, "refseq_category"]) .&
    (ncbi_metadata[!, "genome_size"] .> 10_000), :]

smallest_genomes = DataFrames.DataFrame()
for group_dataframe in DataFrames.groupby(filtered_ncbi_metadata, "group")
    smallest_genome_row = sort(group_dataframe, "genome_size")[1, :]
    push!(smallest_genomes, smallest_genome_row)
end
sort!(smallest_genomes, "genome_size")
selected_genomes = smallest_genomes[1:3, :]

Row,#assembly_accession,bioproject,biosample,wgs_master,refseq_category,taxid,species_taxid,organism_name,infraspecific_name,isolate,version_status,assembly_level,release_type,genome_rep,seq_rel_date,asm_name,asm_submitter,gbrs_paired_asm,paired_asm_comp,ftp_path,excluded_from_refseq,relation_to_type_material,asm_not_live_date,assembly_type,group,genome_size,genome_size_ungapped,gc_percent,replicon_count,scaffold_count,contig_count,annotation_provider,annotation_name,annotation_date,total_gene_count,protein_coding_gene_count,non_coding_gene_count,pubmed_id
,String,String,String,String,String,Int64,Int64,String,String,String,String,String,String,String,Date,String,String,String,String,String,String,String,String,String,String,Int64,Int64,Float64,Int64,Int64,Int64,String,String,Date,Int64,Int64,Int64,String
1,GCF_001610725.1,PRJNA485481,na,na,reference genome,64318,64318,Spondweni virus,strain=SM-6 V-1,na,latest,Complete Genome,Major,Full,2016-02-05,ViralProj311759,Unite des Virus Emergents,GCA_001610725.1,identical,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/001/610/725/GCF_001610725.1_ViralProj311759,na,na,na,haploid,viral,10290,10290,52.5,1,1,1,NCBI RefSeq,Annotation submitted by NCBI RefSeq,2019-08-01,1,1,0,19741066;17195954
2,GCF_024820135.1,PRJNA224116,SAMN26686293,na,representative genome,1160784,1160784,Candidatus Nasuia deltocephalinicola,strain=HICO,na,latest,Chromosome,Major,Full,2022-09-01,ASM2482013v1,University of California Merced,GCA_024820135.1,identical,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/024/820/135/GCF_024820135.1_ASM2482013v1,na,na,na,haploid,bacteria,112590,112299,17.5,1,1,1,NCBI RefSeq,NCBI Prokaryotic Genome Annotation Pipeline (PGAP),2023-12-03,160,133,27,35982793
3,GCF_023169545.1,PRJNA224116,SAMD00173595,na,representative genome,2586965,2586965,Nanobdella aerobiophila,strain=MJ1,na,latest,Complete Genome,Major,Full,2022-02-22,ASM2316954v1,RIKEN,GCA_023169545.1,identical,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/023/169/545/GCF_023169545.1_ASM2316954v1,na,assembly from type material,na,haploid,archaea,668961,668961,24.5,1,1,1,NCBI RefSeq,NCBI Prokaryotic Genome Annotation Pipeline (PGAP),2023-09-20,765,719,46,35993221


In [3]:
data_dir = joinpath(PROJECT_BASEDIR, "data")
genome_dir = mkpath(joinpath(data_dir, "genomes"))
for row in DataFrames.eachrow(selected_genomes)
    records = collect(Mycelia.get_sequence(db="nuccore", ftp=Mycelia.ncbi_ftp_path_to_url(ftp_path=row["ftp_path"], extension="genomic.fna.gz"))) 
    outfile = joinpath(genome_dir, row["#assembly_accession"] * ".fna")
    Mycelia.write_fasta(outfile = outfile, records = records)
end